In [1]:
%matplotlib inline
from os import listdir
from os.path import isdir
from PIL import Image
from matplotlib import pyplot
from numpy import savez_compressed
from numpy import asarray
from mtcnn.mtcnn import MTCNN
from numpy import load
from numpy import expand_dims
from keras.models import load_model
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
from scipy.spatial.distance import cosine
from scipy.spatial import distance
import scipy
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import rcParams
import ast
import tensorflow as tf


In [2]:
directory1 = 'images/known/'
directory2 = 'images/test/'

In [3]:
def read_file():
    file = open("File.txt", "r")
    contents = file.read()
    dictionary = ast.literal_eval(contents)
    file.close()
    return dictionary

In [4]:
def find_images_matching_criteria(dictionary1, criteria):
    known_image = ''
    test_image = ''
    for key, value in dictionary1.items():
        if(key == criteria):
            known_image = value[0]
            test_image = value[1]
    return known_image, test_image

In [5]:
def generating_dataset(known_image, test_image):
    directory1 = 'images/known/'
    directory2 = 'images/test/'
    trainX, trainy = load_dataset(directory1, known_image)
    testX, testy = load_dataset(directory2, test_image)
    savez_compressed('dataset_for_image.npz', trainX, trainy, testX, testy)


In [6]:
def load_dataset(directory, fname1):
    X,y = list(), list()
    path = directory + '/' + fname1 + '.jpg'
    faces = load_faces(path)
    labels = fname1
    X.extend(faces)
    y.extend(labels)
    
    return asarray(X), asarray(y)

def load_faces(directory):
    faces = list()
    face = extract_face(directory)
    faces.append(face)
    return faces

def extract_face(filename, required_size=(160,160)):
    
    image = Image.open(filename)
    image = image.convert('RGB')
    pixels = asarray(image)
    detector = MTCNN()
    results = detector.detect_faces(pixels)
    
    x1, y1, width, height = results[0]['box']
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    
    face = pixels[y1:y2, x1:x2]
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = asarray(image)
    

    return face_array

     

%pylab inline
def plot_initial_images(directory1, directory2, known_image, test_image):
    path_known = directory1 + '/' + known_image + '.jpg'
    path_test = directory2 + '/' + test_image + '.jpg'
    rcParams['figure.figsize'] = 13 ,10
    img_A = mpimg.imread(path_known)
    img_B = mpimg.imread(path_test)
    fig, ax = plt.subplots(1,2)
    ax[0].imshow(img_A);
    ax[1].imshow(img_B);
    


Populating the interactive namespace from numpy and matplotlib


In [8]:
def get_embedding(model, face_pixels):
    face_pixels = face_pixels.astype('float32')
    mean, std = face_pixels.mean(), face_pixels.std()
    face_pixels = (face_pixels - mean) / std
    samples = expand_dims(face_pixels, axis=0)
    yhat = model.predict(samples)
    return yhat[0]

def creating_embeddings():
    data = load('dataset_for_image.npz')
    trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
    model = load_model('facenet_keras.h5')

    newTrainX = list()
    for face_pixels in trainX:
        embedding = get_embedding(model, face_pixels)
        newTrainX.append(embedding)
    newTrainX = asarray(newTrainX)

    newTestX = list()
    for face_pixels in testX:
        embedding = get_embedding(model, face_pixels)
        newTestX.append(embedding)
    newTestX = asarray(newTestX)

    savez_compressed('embeddings_for_images_created.npz', newTrainX, trainy, newTestX, testy)
    print("Embeddings generated")

In [ ]:
def loading_matching():
    data1 = load('embeddings_for_images_created.npz')
    trainX, trainy, testX, testy = data1['arr_0'], data1['arr_1'], data1['arr_2'], data1['arr_3']
    is_match(trainX, testX)
    
def is_match(known_embedding, candidate_embedding, thresh=0.5):
    score = cosine(known_embedding, candidate_embedding)
    if score<=thresh:
        print('Face is a Match (%.3f <= %.3f)' % (score, thresh))
    else:
        print('Face is NOT a Match (%.3f > %.3f)' % (score, thresh))
    cosine_similarity1 = 1 - score
    cosine_similarity = cosine_similarity1 * 100
    print(abs(cosine_similarity))